In [ ]:
import pandas as pd
from global_variables import CSV_BOOK_DATA_PATH
from global_variables import CSV_BOOK_RATING_PATH
from global_variables import CSV_BOOK_DATA_CLEANED_PATH
from global_variables import CSV_BOOK_RATING_CLEANED_PATH
from global_variables import PERCENT